In [1]:
import numpy as np
import pandas as pd
import math
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
warnings.filterwarnings('ignore')
df=pd.read_csv(r"adult_edited.data")

In [2]:
#TASK 1
df = df[df['workclass'] != ' ?']
df = df[df['native-country'] != ' ?']
df_train = df.sample(frac=0.67,random_state=42)
df_test = df.drop(df_train.index)


In [3]:
#TASK 2

#Calculating prior probabilities
df_train_temp = df_train.apply(lambda x : True
            if x['target'] == " <=50K" else False, axis = 1)
num_less = len(df_train_temp[df_train_temp == True].index)
print(num_less)

df_train_temp = df_train.apply(lambda x : True
            if x['target'] == " >50K" else False, axis = 1)
num_grea = len(df_train_temp[df_train_temp == True].index)
print(num_grea)

total = num_less+num_grea
ppl=num_less/total
ppg=num_grea/total
print('Prior probability of <=50K is ',ppl)
print('Prior probability of >50K is ',ppg)

4551
1513
Prior probability of <=50K is  0.7504947229551451
Prior probability of >50K is  0.2495052770448549


In [4]:
#Calculating conditional probabilities of features wrt target

#number of rows that contain arg1
def count_inst_1(arg1): #as this is only for target column, we don't need to worry about wrong column being selected
    for col in df_train.columns:
        if arg1 in df_train[col].values:
            return df_train[col].value_counts()[arg1]
# print(count_instances(' Bachelors'))  not matching ctrl+f of vs code for some reason

#returns the column that contains arg
def whichCol(index):
    if index==0:
        return 'age'
    if index==1:
        return 'workclass'
    if index==2:
        return 'fnlwgt'
    if index==3:
        return 'education'
    if index==4:
        return 'education-num'
    if index==5:
        return 'marital-status'
    if index==6:
        return 'occupation'
    if index==7:
        return 'relationship'
    if index==8:
        return 'race'
    if index==9:
        return 'sex'
    if index==10:
        return 'capital-gain'
    if index==11:
        return 'capital-loss'
    if index==12:
        return 'house-per-week'
    if index==13:
        return 'native-country'
    if index==14:
        return 'target'

#number of rows that contain both arg1 and arg2
def count_inst_2(arg1, arg2,f_index):
    col1 = whichCol(f_index)
    col2 = whichCol(14)
    mask = (df_train[col1].str.contains(arg1)) & (df_train[col2].str.contains(arg2))
    return len(df_train[mask])

#returns conditional probability if the feature is discrete
def condProb_disc(feature,target,f_index):
    x1 = count_inst_2(feature,target,f_index)
    x2 = count_inst_1(target)
    
    return ((x1+1)/(x2+14)) #laplace smoothing applied

def count_normal(feature, target,f_index):
    col1 = whichCol(f_index)
    col2 = 'target'
    #mask = df_train.query("@col2 == @target")
    mask = df_train.loc[df_train[col2] == target]
    mu = mask[col1].mean()
    sigma = mask[col1].std()
    return mu, sigma
    

#returns conditional probability if the feature is continuous
def condProb_cont(feature,target,f_index):
    mu, sigma = count_normal(feature, target,f_index)
    normdistprob = (1/(math.sqrt(2*math.pi)*sigma))*math.exp(-(feature - mu)**2/(2*sigma**2))
#     if(normdistprb == 0):
#         normdistprob = 0.001 #smoothing tactic
    return normdistprob
    
    

#returns conditional probability
def condProb(feature,target,f_index):
    str=whichCol(f_index)
    #print(str)
    if str=='age':
        return condProb_cont(feature,target,f_index)
    if str=='fnlwgt':
        return condProb_cont(feature,target,f_index)
    if str=='education-num':
        return condProb_cont(feature,target,f_index)
    if str=='capital-gain':
        return condProb_cont(feature,target,f_index)
    if str=='capital-loss':
        return condProb_cont(feature,target,f_index)
    if str=='house-per-week':
        return condProb_cont(feature,target,f_index)
    return condProb_disc(feature,target,f_index)

In [5]:
def calcFinalProb(arg1,arg2,arg3,arg4,arg5,arg6,arg7,arg8,arg9,arg10,arg11,arg12,arg13,arg14, target):
    bigterml= condProb(arg1,' <=50K',0)*condProb(arg2,' <=50K',1)*condProb(arg3,' <=50K',2)*condProb(arg4,' <=50K',3)*condProb(arg5,' <=50K',4)*condProb(arg6,' <=50K',5)*condProb(arg7,' <=50K',6)*condProb(arg8,' <=50K',7)*condProb(arg9,' <=50K',8)*condProb(arg10,' <=50K',9)*condProb(arg11,' <=50K',10)*condProb(arg12,' <=50K',11)*condProb(arg13,' <=50K',12)*condProb(arg14,' <=50K',13)
    bigtermg= condProb(arg1,' >50K',0)*condProb(arg2,' >50K',1)*condProb(arg3,' >50K',2)*condProb(arg4,' >50K',3)*condProb(arg5,' >50K',4)*condProb(arg6,' >50K',5)*condProb(arg7,' >50K',6)*condProb(arg8,' >50K',7)*condProb(arg9,' >50K',8)*condProb(arg10,' >50K',9)*condProb(arg11,' >50K',10)*condProb(arg12,' >50K',11)*condProb(arg13,' >50K',12)*condProb(arg14,' >50K',13)
    if target==' <=50K':
        return (ppl*bigterml)/((ppl*bigterml)+(ppg*bigtermg))
    if target==' >50K':
        return (ppg*bigtermg)/((ppl*bigterml)+(ppg*bigtermg))        

In [6]:
def classify(arg1,arg2,arg3,arg4,arg5,arg6,arg7,arg8,arg9,arg10,arg11,arg12,arg13,arg14):
    a=calcFinalProb(arg1,arg2,arg3,arg4,arg5,arg6,arg7,arg8,arg9,arg10,arg11,arg12,arg13,arg14,' <=50K')
    b=calcFinalProb(arg1,arg2,arg3,arg4,arg5,arg6,arg7,arg8,arg9,arg10,arg11,arg12,arg13,arg14,' >50K')
    if(a>b):
        return ' <=50K'
    else:
        return ' >50K'
print (classify(53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', ' Male', 0, 0, 40, ' United-States'))

 <=50K


In [7]:
dftest_y = df_test['target']
df2 = df_test.drop(['target'], axis = 1)

In [8]:
# #NOW WE WANT TO TEST THE ACCURACY USING DF_TEST
# #There are 9953 rows in df_test
# correct=0
# for i in range (9953):
#     str = classify(df_test.iat[i,0],df_test.iat[i,1],df_test.iat[i,2],df_test.iat[i,3],df_test.iat[i,4],df_test.iat[i,5],df_test.iat[i,6],df_test.iat[i,7],df_test.iat[i,8],df_test.iat[i,9],df_test.iat[i,10],df_test.iat[i,11],df_test.iat[i,12],df_test.iat[i,13],df_test.iat[i,14])
#     if str==df_test.iat[i,14]:
#         correct=correct+1
# print('accuracy: ')
# print(correct/9953)
# #taking too long

In [9]:
# Create an empty dataframe to store the results
result_df = pd.DataFrame(columns=['predicted'])

# Initialize the correct counter
correct = 0
tp = 0
tn = 0
fp = 0
fn = 0

#we shall consider >50K as positive and <= 50K as negative

# Iterate over each row of the dataframe using itertuples
for row in df2.itertuples(index=False):
    age, workclass, fnlwgt, education, education_num, marital_status, occupation, \
    relationship, race, sex, capital_gain, capital_loss, hours_per_week, native_country = row
    
    # call classify() function with the row values
    result = classify(age, workclass, fnlwgt, education, education_num, marital_status, 
                      occupation, relationship, race, sex, capital_gain, capital_loss, 
                      hours_per_week, native_country)
    #print(result)
    # Add the result to the result_df
    result_df = result_df.append({'predicted': result}, ignore_index=True)

# Compare the result_df with the dftest_y dataframe
# for index, row in df_test['target'].iteritems():
#     if row == (result_df.loc[index]):
#         correct += 1
   
result_df = result_df.to_numpy()
actual_df = df_test['target'].to_numpy()

for i in range (len(result_df)):
    if result_df[i] == actual_df[i]:
        correct += 1
        if result_df[i] == ' <50k':
            tn += 1
        else:
            tp += 1
    else:
        if result_df[i] == ' <50k':
            fn += 1
        else:
            fp += 1



#Calculate the accuracy
accuracy = correct / len(result_df)
precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1 = tp/(tp + 1/2*(fp+fn))

# Print the accuracy
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall", recall)
print("F1 score", f1)

Accuracy: 0.8302644794107801
Precision: 0.8302644794107801
Recall 1.0
F1 score 0.9072617523321749
